# Érvényesítsük az adatokat

## Derítsük ki, hogy mit rejtenek az adatok

In [1]:
from lxml import objectify
import pandas as pd

xml = objectify.parse(open('XMLAdatok2.xml'))
gyökér = xml.getroot()
ak = pd.DataFrame(columns=('Szám', 'Sztring', 'Logikai'))

for i in range(0,4):
    obj = gyökér.getchildren()[i].getchildren()
    sor = dict(zip(['Szám', 'Sztring', 'Logikai'],
                   [obj[0].text, obj[1].text,
                    obj[2].text]))
    sor_s = pd.Series(sor)
    sor_s.name = i
    sor_s = sor_s.to_frame().transpose()
    ak = pd.concat([ak, sor_s])

keresés = pd.DataFrame.duplicated(ak)
print(ak)
print(f"\n{keresés[keresés == True]}")

  Szám   Sztring Logikai
0    1      Első    Igaz
1    2   Második   Hamis
2    3  Harmadik    Igaz
3    3  Harmadik    Igaz

3    True
dtype: bool


## Távolítsuk el a duplikátumokat

In [2]:
from lxml import objectify
import pandas as pd

xml = objectify.parse(open('XMLAdatok2.xml'))
gyökér = xml.getroot()
df = pd.DataFrame(columns=('Szám', 'Sztring', 'Logikai'))
for i in range(0,4):
    obj = gyökér.getchildren()[i].getchildren()
    sor = dict(zip(['Szám', 'Sztring', 'Logikai'],
                   [obj[0].text, obj[1].text,
                    obj[2].text]))
    sor_s = pd.Series(sor)
    sor_s.name = i
    sor_s = sor_s.to_frame().transpose()
    ak = pd.concat([ak, sor_s])

print(ak.drop_duplicates())

  Szám   Sztring Logikai
0    1      Első    Igaz
1    2   Második   Hamis
2    3  Harmadik    Igaz


## Készítsünk adattérképet és adattervet

In [3]:
import pandas as pd
pd.set_option('display.width', 55)

ak = pd.DataFrame({'A': [0,0,0,0,0,1,1],
                   'B': [1,2,3,5,4,2,5],
                   'C': [5,3,4,1,1,2,3]})

a_csoportleírás = ak.groupby('A').describe()
print(a_csoportleírás)

      B                                            \
  count mean       std  min   25%  50%   75%  max   
A                                                   
0   5.0  3.0  1.581139  1.0  2.00  3.0  4.00  5.0   
1   2.0  3.5  2.121320  2.0  2.75  3.5  4.25  5.0   

      C                                            
  count mean       std  min   25%  50%   75%  max  
A                                                  
0   5.0  2.8  1.788854  1.0  1.00  3.0  4.00  5.0  
1   2.0  2.5  0.707107  2.0  2.25  2.5  2.75  3.0  


In [4]:
halmozott = a_csoportleírás.stack()
print(halmozott)

                B         C
A                          
0 count  5.000000  5.000000
  mean   3.000000  2.800000
  std    1.581139  1.788854
  min    1.000000  1.000000
  25%    2.000000  1.000000
  50%    3.000000  3.000000
  75%    4.000000  4.000000
  max    5.000000  5.000000
1 count  2.000000  2.000000
  mean   3.500000  2.500000
  std    2.121320  0.707107
  min    2.000000  2.000000
  25%    2.750000  2.250000
  50%    3.500000  2.500000
  75%    4.250000  2.750000
  max    5.000000  3.000000


In [5]:
print(a_csoportleírás.loc[:,(slice(None),['count','mean']),])

      B          C     
  count mean count mean
A                      
0   5.0  3.0   5.0  2.8
1   2.0  3.5   2.0  2.5


# Kezeljünk kategorikus változókat

## A pandas verziójának ellenőrzése

In [6]:
import pandas as pd
print(pd.__version__)

1.5.3


## Hozzunk létre kategorikus változókat

In [7]:
import pandas as pd

autószínek = pd.Series(['Kék', 'Piros', 'Zöld'],
                       dtype='category')
autóadatok = pd.Series(
    pd.Categorical(
        ['Sárga', 'Zöld', 'Piros', 'Kék', 'Lila'],
        categories = autószínek, ordered = False))

bejegyzések_keresése = pd.isnull(autóadatok)

print(autószínek)
print(f"\n{autóadatok}")
print(f"\n{bejegyzések_keresése[bejegyzések_keresése == True]}")

0      Kék
1    Piros
2     Zöld
dtype: category
Categories (3, object): ['Kék', 'Piros', 'Zöld']

0      NaN
1     Zöld
2    Piros
3      Kék
4      NaN
dtype: category
Categories (3, object): ['Kék', 'Piros', 'Zöld']

0    True
4    True
dtype: bool


## Nevezzünk át szinteket

In [8]:
import pandas as pd

autószínek = pd.Series(['Kék', 'Piros', 'Zöld'],
                       dtype='category')
autóadatok = pd.Series(
    pd.Categorical(
        ['Kék', 'Zöld', 'Piros', 'Kék', 'Piros'],
        categories=autószínek, ordered=False))

autóadatok = autóadatok.cat.rename_categories(
    ["Lila", "Sárga", "Mályva"])

print(autóadatok)

0      Lila
1    Mályva
2     Sárga
3      Lila
4     Sárga
dtype: category
Categories (3, object): ['Lila', 'Sárga', 'Mályva']


## Vonjunk össze szinteket

In [9]:
import pandas as pd

autószínek = pd.Series(['Kék', 'Piros', 'Zöld'],
    dtype='category')
autóadatok = pd.Series(
    pd.Categorical(
       ['Kék', 'Zöld', 'Piros', 'Zöld', 'Piros', 'Zöld'],
       categories=autószínek, ordered=False))
autóadatok = autóadatok.cat.set_categories(
    ["Kék", "Piros", "Zöld", "Kék_Piros"])

print(autóadatok.loc[autóadatok.isin(['Piros'])])

autóadatok.loc[autóadatok.isin(['Piros'])] = 'Kék_Piros'
autóadatok.loc[autóadatok.isin(['Kék'])] = 'Kék_Piros'
autóadatok = autóadatok.cat.set_categories(
    ["Zöld", "Kék_Piros"])

print(f"\n{autóadatok}")

2    Piros
4    Piros
dtype: category
Categories (4, object): ['Kék', 'Piros', 'Zöld', 'Kék_Piros']

0    Kék_Piros
1         Zöld
2    Kék_Piros
3         Zöld
4    Kék_Piros
5         Zöld
dtype: category
Categories (2, object): ['Zöld', 'Kék_Piros']


# Kezeljük a dátumokat az adatokban

## Formázzunk dátum- és időértékeket

In [10]:
import datetime as di

most = di.datetime.now()

print(str(most))
print(most.strftime('%a, %d %B %Y'))

2024-03-14 12:19:53.730175
Thu, 14 March 2024


## Használjuk a megfelelő időtranszformációt

In [11]:
import datetime as di

most = di.datetime.now()
időérték = most + di.timedelta(hours=2)

print(most.strftime('%H:%M:%S'))
print(időérték.strftime('%H:%M:%S'))
print(időérték - most)

12:19:53
14:19:53
2:00:00


# Kezeljük a hiányzó adatokat

## Keressük meg a hiányzó adatokat

In [12]:
import pandas as pd
import numpy as np

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])

print(s.isnull())
print(f"\n{s[s.isnull()]}")

0    False
1    False
2    False
3     True
4    False
5    False
6     True
dtype: bool

3   NaN
6   NaN
dtype: float64


## Kódoljuk a hiányt

In [13]:
import pandas as pd
import numpy as np

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])

print(s.fillna(int(s.mean())))
print(f"\n{s.dropna()}")

0    1.0
1    2.0
2    3.0
3    3.0
4    5.0
5    6.0
6    3.0
dtype: float64

0    1.0
1    2.0
2    3.0
4    5.0
5    6.0
dtype: float64


## Rendeljünk hozzá hiányzó adatokat

In [14]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

s = pd.DataFrame([1, 2, 3, np.nan, 5, 6, np.nan])

imp = SimpleImputer(missing_values=np.nan,
                    add_indicator=True,
                    strategy='mean')

imp.fit(s)
x = imp.transform(s)
print(x)

[[1.  0. ]
 [2.  0. ]
 [3.  0. ]
 [3.4 1. ]
 [5.  0. ]
 [6.  0. ]
 [3.4 1. ]]


# Egy- és többirányú szeletelés: szűrjük és jelöljük ki az adatokat

## Szeleteljünk sorokat

In [15]:
x = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
             [[11,12,13], [14,15,16], [17,18,19],],
             [[21,22,23], [24,25,26], [27,28,29]]])
x[1]

array([[11, 12, 13],
       [14, 15, 16],
       [17, 18, 19]])

## Szeleteljünk oszlopokat

In [16]:
x = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
             [[11,12,13], [14,15,16], [17,18,19],],
             [[21,22,23], [24,25,26], [27,28,29]]])
x[:,1]

array([[ 4,  5,  6],
       [14, 15, 16],
       [24, 25, 26]])

## Végezzünk többirányú szeletelést

In [17]:
x = np.array([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
             [[11,12,13], [14,15,16], [17,18,19],],
             [[21,22,23], [24,25,26], [27,28,29]]])
print(x[1,1])
print(x[:,1,1])
print(x[1,:,1])
print(f"\n{x[1:2, 1:2]}")

[14 15 16]
[ 5 15 25]
[12 15 18]

[[[14 15 16]]]


# Fűzzünk össze és transzformáljunk adatokat

## Adjunk hozzá új eseteket és változókat

In [18]:
import pandas as pd

ak = pd.DataFrame({'A': [2,3,1],
                   'B': [1,2,3],
                   'C': [5,3,4]})

ak1 = pd.DataFrame({'A': [4],
                    'B': [4],
                    'C': [4]})

ak = pd.concat([ak, ak1])
ak = ak.reset_index(drop=True)
print(ak)

ak.loc[ak.last_valid_index() + 1] = [5, 5, 5]
print(f"\n{ak}")

ak2 = pd.DataFrame({'D': [1, 2, 3, 4, 5]})

ak = pd.DataFrame.join(ak, ak2)
print(f"\n{ak}")

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4
4  5  5  5

   A  B  C  D
0  2  1  5  1
1  3  2  3  2
2  1  3  4  3
3  4  4  4  4
4  5  5  5  5


## Távolítsunk el adatokat

In [19]:
import pandas as pd

ak = pd.DataFrame({'A': [2,3,1],
                   'B': [1,2,3],
                   'C': [5,3,4]})

ak = ak.drop(ak.index[[1]])
print(ak)

ak = ak.drop(columns=['B'])
print(f"\n{ak}")

   A  B  C
0  2  1  5
2  1  3  4

   A  C
0  2  5
2  1  4


## Rendezzük és keverjük össze az adatokat

In [20]:
import pandas as pd
import numpy as np

ak = pd.DataFrame({'A': [2,1,2,3,3,5,4],
                   'B': [1,2,3,5,4,2,5],
                   'C': [5,3,4,1,1,2,3]})

ak = ak.sort_values(by=['A', 'B'], ascending=[True, True])
ak = ak.reset_index(drop=True)
print(ak)

index = ak.index.tolist()
np.random.shuffle(index)
ak = ak.loc[ak.index[index]]
ak = ak.reset_index(drop=True)
print(f"\n{ak}")

   A  B  C
0  1  2  3
1  2  1  5
2  2  3  4
3  3  4  1
4  3  5  1
5  4  5  3
6  5  2  2

   A  B  C
0  4  5  3
1  3  5  1
2  3  4  1
3  2  3  4
4  2  1  5
5  1  2  3
6  5  2  2


# Összesítsük az adatokat tetszőleges szinten

In [21]:
import pandas as pd

ak = pd.DataFrame({'Hozzárendelés': [0,0,0,1,1,2,2],
                   'Értékek': [1,2,3,5,4,2,5]})

ak['Ö'] = ak.groupby('Hozzárendelés')['Értékek'].transform(np.sum)
ak['Á'] = ak.groupby('Hozzárendelés')['Értékek'].transform(np.mean)
ak['Sz'] = ak.groupby('Hozzárendelés')['Értékek'].transform(np.var)

print(ak)

   Hozzárendelés  Értékek  Ö    Á   Sz
0              0        1  6  2.0  1.0
1              0        2  6  2.0  1.0
2              0        3  6  2.0  1.0
3              1        5  9  4.5  0.5
4              1        4  9  4.5  0.5
5              2        2  7  3.5  4.5
6              2        5  7  3.5  4.5
